In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heartbeat/ptbdb_abnormal.csv
/kaggle/input/heartbeat/ptbdb_normal.csv
/kaggle/input/heartbeat/mitbih_test.csv
/kaggle/input/heartbeat/mitbih_train.csv


In [20]:
import matplotlib.pyplot as plt
import seaborn as sns
import copy

import pywt
import scipy

from skimage.restoration import denoise_wavelet

import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans

from tensorflow.keras.layers import Layer, Dense, Dropout, BatchNormalization
from tensorflow.keras.initializers import RandomUniform, Initializer, Constant

from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report

from keras.utils.np_utils import to_categorical

from keras.optimizers import RMSprop,Adam
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

In [21]:
train_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header=None)
test_df=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)

In [22]:
train_x = np.array(train_df[train_df.columns[0:-1]], dtype=np.float32)
train_y = np.array(train_df[train_df.columns[-1]], dtype=np.float32)

test_x = np.array(test_df[test_df.columns[0:-1]], dtype=np.float32)
test_y = np.array(test_df[test_df.columns[-1]], dtype=np.float32)

In [23]:
c=train_df.groupby(187,group_keys=False).apply(lambda train_df : train_df.sample(1))
cind=c.index.to_numpy()
cind

array([11535, 74084, 78064, 80939, 81935])

In [24]:
for i in range(len(train_x)):
    train_x[i] =  denoise_wavelet(train_x[i] ,method='BayesShrink', mode='soft',
                           rescale_sigma=True, wavelet='sym8', wavelet_levels=3)

for i in range(len(test_x)):
    test_x[i] =  denoise_wavelet(test_x[i] ,method='BayesShrink', mode='soft',
                           rescale_sigma=True, wavelet='sym8', wavelet_levels=3)

In [25]:
def wpd(x):

    Xfeatures=np.zeros((len(x),8,29))

    for i in range(len(x)):
        wpack=pywt.WaveletPacket(data=x[i], wavelet='db4')
        Xfeatures[i]=np.vstack((wpack['aaa'].data,wpack['add'].data,wpack['dad'].data,wpack['dda'].data,
                                wpack['aad'].data,wpack['daa'].data,wpack['ada'].data,wpack['ddd'].data))
    
    return Xfeatures

In [26]:
Xfeatures=wpd(train_x)

In [27]:
def scaling(x):
    sc=MinMaxScaler([0,1])
    x=x.reshape(len(x),len(x[0]),len(x[0][0]),1)
    scaled=np.zeros((len(x),len(x[0]),len(x[0][0]),1))
    #x=x.reshape(len(x),len(x[0]),len(x[0][0]))
    
    for i in range(len(x)):
        for j in range(8):
            sc.fit(x[i][j])
            scaled[i][j]=sc.transform(x[i][j])
            
    return scaled

In [28]:
Xfeat_scaled = scaling(Xfeatures)
Xfeat_scaled = Xfeat_scaled.reshape(-1,8,29)

In [29]:
def wps(x):
    Xps= np.zeros((len(x),len(x[0]),len(x[0][0])))
    Xband_energy= np.zeros((len(x),len(x[0])))
        
    for i in range(len(x)):
        Xps[i]=np.square(np.abs(x[i]))        
        
    return Xps

In [30]:
Xscaled_power_spectrum=wps(Xfeat_scaled)

In [31]:
Xapprox = Xscaled_power_spectrum[:,0,:]
X_train, X_test, y_train, y_test = train_test_split(Xapprox, train_y, test_size=0.25, random_state=42, stratify=train_y)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=5) 

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (65665, 29)
Number transactions y_train dataset:  (65665, 5)
Number transactions X_test dataset:  (21889, 29)
Number transactions y_test dataset:  (21889, 5)


In [32]:
class InitCentersKMeans(Initializer):
    """ Initializer for initialization of centers of RBF network
        by clustering the given data set.
    # Arguments
        X: matrix, dataset
    """

    def __init__(self, X, max_iter=100):
        self.X = X
        self.max_iter = max_iter
        super().__init__()

    def __call__(self, shape, dtype=None):
        #assert shape[0:] == self.X.shape[1:]

        n_centers = shape[0]
        km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0, init='k-means++')
        km.fit(self.X)
        return km.cluster_centers_

In [33]:
class RBFLayer(Layer):

    def __init__(self, output_dim, initializer=None, betas=2.0, **kwargs):

        self.output_dim = output_dim

        # betas is either initializer object or float
        if isinstance(betas, Initializer):
            self.betas_initializer = betas
        else:
            self.betas_initializer = Constant(value=betas)

        self.initializer = initializer if initializer else RandomUniform(
            0.0, 1.0)

        super().__init__(**kwargs)

    def build(self, input_shape):

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.output_dim,),
                                     initializer=self.betas_initializer,
                                     # initializer='ones',
                                     trainable=True)

        super().build(input_shape)

    def call(self, x):

        C = tf.expand_dims(self.centers, -1)  # inserts a dimension of 1
        H = tf.transpose(C-tf.transpose(x))  # matrix of differences
        return tf.exp(-self.betas * tf.math.reduce_sum(H**2, axis=1))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        # have to define get_config to be able to use model_from_json
        config = {
            'output_dim': self.output_dim
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [34]:
rbflayer = RBFLayer(232,
                    initializer=InitCentersKMeans(X_train),
                    betas=1.0,
                    input_shape=(29,))
#-----Input layer------#

inputs=tf.keras.layers.Input(shape=(29,))

#-----RBF layer------#

x=rbflayer(inputs)

x=tf.keras.layers.Dropout(0.1,name='drop4')(x)
x=Dense(800, activation='linear', name='dense_2')(x)
x=BatchNormalization()(x)
#x=tf.keras.layers.Dropout(0.1,name='drop5')(x)
x=tf.keras.layers.Reshape((25,32), name='reshape1')(x)

#----ConvNet Layers-----#

x=tf.keras.layers.Conv1D(64,5, activation='relu', name='conv1')(x)
x=BatchNormalization()(x)
x=tf.keras.layers.Conv1D(128,3, activation='relu', name='conv2')(x)
x=BatchNormalization()(x)
x=tf.keras.layers.Conv1D(256,3, activation='relu', name='conv3')(x)
x=BatchNormalization()(x)

x=tf.keras.layers.Dropout(0.2,name='drop5')(x)


x=tf.keras.layers.Flatten()(x)
x=Dense(512, activation='relu', name='dense_4')(x)
x=BatchNormalization()(x)
x=Dense(256, activation='relu', name='dense_5')(x)
x=tf.keras.layers.Dropout(0.3,name='drop6')(x)
x=Dense(128, activation='relu', name='dense_6')(x)
x=BatchNormalization()(x)

x=Dense(5, activation='softmax', name='output')(x)
outputs=x


model2 = tf.keras.Model(inputs= inputs, outputs=outputs)
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 29)]              0         
_________________________________________________________________
rbf_layer_1 (RBFLayer)       (None, 232)               6960      
_________________________________________________________________
drop4 (Dropout)              (None, 232)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 800)               186400    
_________________________________________________________________
batch_normalization_6 (Batch (None, 800)               3200      
_________________________________________________________________
reshape1 (Reshape)           (None, 25, 32)            0         
_________________________________________________________________
conv1 (Conv1D)               (None, 21, 64)            1030

In [35]:
model2.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer='Adam',
                 metrics=['accuracy'])

annealer = LearningRateScheduler(lambda x: 1e-2 * 0.9 ** x)


history=model2.fit(X_train, y_train,
              batch_size=64,
              epochs=100,
                  steps_per_epoch=len(X_train)//64,
              validation_data=(X_test,y_test),
                  callbacks=[annealer])

Epoch 1/100
1026/1026 [==============================] - 11s 9ms/step - loss: 0.4023 - accuracy: 0.8894 - val_loss: 0.1963 - val_accuracy: 0.9494
Epoch 2/100
1026/1026 [==============================] - 9s 8ms/step - loss: 0.2265 - accuracy: 0.9411 - val_loss: 0.2180 - val_accuracy: 0.9434
Epoch 3/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.1913 - accuracy: 0.9488 - val_loss: 0.1776 - val_accuracy: 0.9539
Epoch 4/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.1790 - accuracy: 0.9526 - val_loss: 0.1668 - val_accuracy: 0.9550
Epoch 5/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.1636 - accuracy: 0.9562 - val_loss: 0.1522 - val_accuracy: 0.9566
Epoch 6/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.1652 - accuracy: 0.9555 - val_loss: 0.1492 - val_accuracy: 0.9616
Epoch 7/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.1514 - accuracy: 0.9581 - val_loss: 0.1306 - val_a

1026/1026 [==============================] - 8s 8ms/step - loss: 0.0430 - accuracy: 0.9858 - val_loss: 0.0827 - val_accuracy: 0.9796
Epoch 58/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.0476 - accuracy: 0.9836 - val_loss: 0.0827 - val_accuracy: 0.9796
Epoch 59/100
1026/1026 [==============================] - 9s 9ms/step - loss: 0.0441 - accuracy: 0.9855 - val_loss: 0.0828 - val_accuracy: 0.9796
Epoch 60/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.0436 - accuracy: 0.9857 - val_loss: 0.0828 - val_accuracy: 0.9798
Epoch 61/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.0487 - accuracy: 0.9838 - val_loss: 0.0830 - val_accuracy: 0.9798
Epoch 62/100
1026/1026 [==============================] - 8s 8ms/step - loss: 0.0468 - accuracy: 0.9845 - val_loss: 0.0829 - val_accuracy: 0.9797
Epoch 63/100
1026/1026 [==============================] - 9s 9ms/step - loss: 0.0469 - accuracy: 0.9847 - val_loss: 0.0830 - val_accuracy

In [36]:
Xtest_feats = wpd(test_x)
Xtest_scaled = scaling(Xtest_feats)
Xtest_scaled = Xtest_scaled.reshape(-1,8,29)

Xtest_scaled_power_spectrum= wps(Xtest_scaled)
X_tested = Xtest_scaled_power_spectrum

In [37]:
X_tested_approx = X_tested[:,0,:]
test_y3 = tf.keras.utils.to_categorical(test_y, num_classes=5) 

ypred3 = model2.predict(X_tested_approx)

for i in range(len(ypred3)):
    ind=np.argmax(ypred3[i])
    for j in range(5):
        if j==ind:
            ypred3[i][j]=1.
        else:
            ypred3[i][j]=0.
            


target_names = ['class 0', 'class 1', 'class 2',  'class 3',  'class 4']
print(classification_report(test_y3, ypred3, target_names=target_names, zero_division='warn'))

              precision    recall  f1-score   support

     class 0       0.98      0.99      0.99     18118
     class 1       0.88      0.69      0.78       556
     class 2       0.94      0.93      0.93      1448
     class 3       0.80      0.70      0.75       162
     class 4       0.98      0.96      0.97      1608

   micro avg       0.98      0.98      0.98     21892
   macro avg       0.92      0.86      0.88     21892
weighted avg       0.98      0.98      0.98     21892
 samples avg       0.98      0.98      0.98     21892



In [39]:
model2.evaluate(X_tested_approx, test_y3)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1217 run_step  **
        outputs = model